## Model Selection
In the previous notebook, we saw how increasing the degree of the polynomial features (i.e. increasing the complexity of the model) lead to a better fit of the (non-linear) data. In this notebook, we will explore what happens when we increase the degree of the polynomial features (i.e. the complexity of the model) even further, and will examine the impact this has on the predictive performance of the regression model. 

Tuning the degree of the polynomial, is an example of *model selection* which is how we describe the method of tuning the parameters of a model to improve performance.

Let's imprt the required packages.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
%matplotlib inline

### Overfitting training data
For this example, we are going to use a smaller dataset with a lower number of samples.

Let's load the data and create a simple scatter plot.

In [ ]:
df_nonlinear = pd.read_csv('Data/nonlinear_small.csv', sep=',', header=0)
df_nonlinear.plot.scatter(x='X', y='y');

Now let's prepare the data, split into a 50:50 train and test split, and visualise the training data with black circles and the test data with blue crosses.

In [ ]:
# Prepare data
y = df_nonlinear['y']
X = df_nonlinear['X'][:, np.newaxis]
X=(X-X.mean())/X.std()

# Split into 50:50 training and testing
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.5, random_state=0)

# Visualise both
ax = pd.DataFrame({'x':X_train[:,0],'y':y_train}).plot.scatter(x='x', y='y',c='k');
pd.DataFrame({'x':X_test[:,0],'y':y_test}).plot.scatter(x='x', y='y',ax=ax,c='b',marker='x');

Now let us create polynomial features using a polynomial feature map of degree $5$. Then let's train a linear regression model on the training data, and visualise the regression model using predictions on a linear space of $X$.

In [ ]:
# Create polynomial features of degree 5
degree = 5
poly = PolynomialFeatures(degree)
X_poly = poly.fit_transform(X_train)

# Fit linear regression model
lmod = LinearRegression()
lmod.fit(X_poly, y_train)

# Create lin space for plotting regression model
X_linspace = np.linspace(np.min(X), np.max(X), num=50)[:, np.newaxis]
y_linspace = lmod.predict(poly.transform(X_linspace))

# Visualise everything
ax = pd.DataFrame({'x':X_train[:,0],'y':y_train}).plot.scatter(x='x', y='y',c='k')
pd.DataFrame({'x':X_test[:,0],'y':y_test}).plot.scatter(x='x', y='y',ax=ax ,c='b',marker='x')
pd.DataFrame({'x':X_linspace[:,0],'y':y_linspace}).plot(x='x', y='y',ax=ax ,c='gray');

How well do you think the above regression model fits the training data (black dots)? How well do you think the above regression fits the test data (blue crosses)?

What is happening in the above figure is a case of *over fitting*. Indeed, we could increase the degree of the model further until we fit all the training data points exactly, but this would likely lead to a large error on the test set.

We can avoid over-fitting the training data, but the problem becomes choosing the degree of the polynomial feature so that the model performs best on new data. 

Let us try this by training the model on the training data for different degrees of polynomial, testing the resulting model on the test data, and storing the RMSE of each model so we can analyse the performance of each model after the experiment.

### Model selection for polynomial degree
Let's now explore model selection in the context of choosing the degree of the polynomial feature map.

First, let's specify the degrees to be tested and create an array for storing the RMSE for each model

In [ ]:
degrees = [0, 1, 2, 3, 4, 5, 6]
rmses = np.zeros((len(degrees),))

Next, let's iterate over the different degrees, and for each degree:
* train the model on the training set
* test the model on the test set
* store the RMSE of the test set

In [ ]:
for degree in degrees:
    poly = PolynomialFeatures(degree)
    poly.fit(X_train)
    
    lmod = LinearRegression()
    lmod.fit(poly.transform(X_train), y_train)
    
    y_pred = lmod.predict(poly.transform(X_test))
    rmses[degree] = (np.sqrt(mean_squared_error(y_test, y_pred)))

Finally, let's plot the degree versus the RMSE to see which degree of polynomial performs best on the test set.

In [ ]:
pd.DataFrame({'degree':degrees,'rmse':rmses}).set_index('degree')['rmse'].plot();

Looks like the model with polynomial degree of $4$ has the lowest RMSE on this particular test set. 

Let's plot predictions of for the *best* regression model, to see if the result looks any better than the previous model with polynomial degree of $5$.

In [ ]:
degree = 4
poly = PolynomialFeatures(degree)
X_poly = poly.fit_transform(X_train)

# Fit linear regression model
lmod = LinearRegression()
lmod.fit(X_poly, y_train)

# Create lin space for plotting regression model
X_linspace = np.linspace(np.min(X), np.max(X), num=50)[:, np.newaxis]
y_linspace = lmod.predict(poly.transform(X_linspace))


ax = pd.DataFrame({'x':X_train[:,0],'y':y_train}).plot.scatter(x='x', y='y',c='k')
pd.DataFrame({'x':X_test[:,0],'y':y_test}).plot.scatter(x='x', y='y',ax=ax ,c='b',marker='x')
pd.DataFrame({'x':X_linspace[:,0],'y':y_linspace}).plot(x='x', y='y',ax=ax ,c='gray');

What do you think? Better than the model with polynomial degree $5$?